In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pickle

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

In [ ]:
items=pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/items.csv")
shops=pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/shops.csv")
cats=pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv")
sales=pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv")
test=pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/test.csv")

In [ ]:
from itertools import product
matrix = []

months = sales.date_block_num.unique()
for month in months:
    mon_sales = sales[sales.date_block_num == month]
    unq_shop = sales['shop_id'].unique()
    unq_item = sales['item_id'].unique()
    append_arr = np.array(list(product(unq_shop, unq_item, [month])), dtype='int16')
    matrix.append(append_arr)

In [ ]:
cols = ['shop_id','item_id', 'date_block_num']
matrix = pd.DataFrame(np.vstack(matrix), columns=cols)
matrix.head()

In [ ]:
matrix['shop_id'] = matrix['shop_id'].astype(np.int8)
matrix['item_id'] = matrix['item_id'].astype(np.int16)
matrix['date_block_num'] = matrix['date_block_num'].astype(np.int8)
matrix.sort_values(['date_block_num','shop_id','item_id'],inplace=True)
matrix.head()

In [ ]:
sales['revenue'] = sales['item_price'] * sales['item_cnt_day']
sales.head()

group_data = sales.groupby(by=['date_block_num', 'shop_id', 'item_id']).agg({'item_cnt_day': 'sum'})
group_data.columns = ['item_cnt_month']
group_data.reset_index(inplace = True)
group_data.head()

cols = ['shop_id','item_id', 'date_block_num']
matrix = pd.merge(matrix, group_data, on=cols, how='left')
matrix.head()

In [ ]:
matrix['item_cnt_month'] = (matrix['item_cnt_month'].fillna(0).clip(0, 20).astype(np.float16))
matrix.head()

In [ ]:
group_data = sales.groupby(by=['date_block_num', 'shop_id', 'item_id']).agg({'revenue': 'sum'})
group_data.columns = ['revenue']
group_data.reset_index(inplace = True)
group_data.head()

cols = ['shop_id','item_id', 'date_block_num']
matrix = pd.merge(matrix, group_data, on=cols, how='left')
matrix.head()

In [ ]:
matrix['revenue'] = (matrix['revenue'].fillna(0).clip(0, 20).astype(np.float16))
matrix.head()

In [ ]:
test['date_block_num'] = 34
test['date_block_num'] = test['date_block_num'].astype(np.int8)
test['shop_id'] = test['shop_id'].astype(np.int8)
test['item_id'] = test['item_id'].astype(np.int16)

matrix = pd.concat([matrix, test], ignore_index=True, sort=False, keys=cols)
matrix.fillna(0, inplace=True)
matrix = matrix.drop('ID', axis=1)
matrix.head()

In [ ]:
matrix.info()

In [ ]:
matrix.describe()

In [ ]:
items.drop('item_name', axis = 1, inplace = True)

In [ ]:
def listToString(s):  
    
    # initialize an empty string 
    str1 = " " 
    
    # return string
    return (str1.join(s))

import re
for i, shop in enumerate(shops['shop_name']):
    if i == 6:
        pass
    else:
        string = shop
        cleanString = re.sub(r'[?|$|.|!]',r'',string)
        splitStr = cleanString.split()[:-1]
        finalStr = listToString(splitStr)
        result = shops[shops['shop_name'].str.contains(finalStr)]
        if result.shape == (2, 2) or result.shape == (0, 2):
            print(result)
            print('-'*50)

In [ ]:
shops['city'] = shops['shop_name'].str.split(' ').map(lambda x:x[0])

In [ ]:
from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder()
shops['city_code'] = enc.fit_transform(shops['city'])

In [ ]:
shops = shops[['shop_id','city_code']]
shops.head()

In [ ]:
matrix = pd.merge(matrix, shops, on=['shop_id'], how='left')
matrix = pd.merge(matrix, items, on=['item_id'], how='left')
matrix = pd.merge(matrix, cats, on=['item_category_id'], how='left')

In [ ]:
matrix.drop('item_category_name', axis = 1, inplace = True)

In [ ]:
group_data = sales.groupby(by=['item_id']).agg({'item_price': ['mean','min', 'max']})
group_data.columns = ['avg_item_price', 'min_item_price', 'max_item_price']
group_data.reset_index(inplace = True)
group_data.head()

In [ ]:
matrix = pd.merge(matrix, group_data, on=['item_id'], how='left')

In [ ]:
matrix['avg_item_price'] = matrix['avg_item_price'].astype('float32')
matrix['min_item_price'] = matrix['min_item_price'].astype('float32')
matrix['max_item_price'] = matrix['max_item_price'].astype('float32')

In [ ]:
matrix['avg_item_price'].fillna(matrix['avg_item_price'].mean(), inplace = True)
matrix['min_item_price'].fillna(matrix['min_item_price'].mean(), inplace = True)
matrix['max_item_price'].fillna(matrix['max_item_price'].mean(), inplace = True)

In [ ]:
matrix

In [ ]:
group = matrix.groupby('shop_id').agg({'item_cnt_month':'mean'})
group.columns = ['shop_avg_item_cnt']
group.reset_index(inplace=True)
matrix = pd.merge(matrix, group, on=['shop_id'], how='left')

group = matrix.groupby('item_id').agg({'item_cnt_month':'mean'})
group.columns = ['item_avg_count']
group.reset_index(inplace=True)
matrix = pd.merge(matrix, group, on=['item_id'], how='left')

group = matrix.groupby(['item_id', 'shop_id']).agg({'item_cnt_month':'mean'})
group.columns = ['item_avg_count_shop']
group.reset_index(inplace=True)
matrix = pd.merge(matrix, group, on=['item_id', 'shop_id'], how='left')

matrix.info()

In [ ]:
group = matrix.groupby(['item_id', 'date_block_num']).agg({'item_cnt_month':'mean'})
group.columns = ['item_avg_count_month']
group.reset_index(inplace=True)
matrix = pd.merge(matrix, group, on=['item_id', 'date_block_num'], how='left')

group = matrix.groupby(['item_category_id']).agg({'item_cnt_month':'mean'})
group.columns = ['item_avg_count_category']
group.reset_index(inplace=True)
matrix = pd.merge(matrix, group, on=['item_category_id'], how='left')

In [ ]:
matrix['month'] = matrix['date_block_num']%12 +1

In [ ]:
matrix.info()

In [ ]:
import gc
import subprocess

In [ ]:
del(group)
del(group_data)
gc.collect()

In [ ]:
del(sales)
del(items)
del(cats)
del(test)
del(shops)
gc.collect()

In [ ]:
cols = ['shop_id',
 'item_id',
 'date_block_num',
 'revenue',
 'city_code',
 'item_category_id',
 'avg_item_price',
 'min_item_price',
 'max_item_price',
 'shop_avg_item_cnt',
 'item_avg_count',
 'item_avg_count_shop',
 'item_avg_count_month',
 'item_avg_count_category',
 'month',
'item_cnt_month',]
matrix = matrix[cols]

In [ ]:
matrix.info(memory_usage='deep')

In [ ]:
matrix['city_code'] = matrix['city_code'].astype('int8')
matrix['item_category_id'] = matrix['item_category_id'].astype('int8')
matrix['avg_item_price'] = matrix['avg_item_price'].astype('float16')
matrix['min_item_price'] = matrix['min_item_price'].astype('float16')
matrix['max_item_price'] = matrix['max_item_price'].astype('float16')

In [ ]:
#matrix.to_csv('sales_data.csv')

In [ ]:
import pickle
matrix.to_pickle('data.pkl') 
gc.collect()

In [ ]:
import psutil
import os
print(u'memory：{}gb'.format(psutil.Process(os.getpid()).memory_info().rss / 1024 / 1024 / 1024))

In [ ]:
del(matrix)
gc.collect()